In [1]:
import respy as rp
import numpy as np
from estimagic.optimization.optimize import maximize

In [2]:
# Parameters for the simulation --> will go to config.py
NUM_AGENTS = 500
NUM_PERIODS = 5

### Working Step 1: Model estimation via Maximum Likelihood

- We estimate a model by Maximum Likelihood estimation using all true values as the starting point. We start with a model that does not include ambiguity, $\eta = 0$. 
- We do not fix any of the values and use a gradient-free optimizer.

In [3]:
params_base, options = rp.get_example_model("robinson_crusoe_basic", with_data=False)

We will simulate NUM_AGENTS representative Robinson Crusoe agents over a time of T = NUM_PERIODS periods.

In [4]:
options["n_periods"] = NUM_PERIODS
options["simulation_agents"] = NUM_AGENTS

In [5]:
simulate = rp.get_simulate_func(params_base, options)
df = simulate(params_base)
#df

**Get the criterion function** and output the value of the MLE at given parameter values.

**Note**  
We obtain a version of the likelihood function where all arguments - except for the parameter vector - are fixed. 
Consequently, the function can be directly passed into an optimizer.

$\eta$ enters through the `model_processing._parse_parameters` method, which was added through [PR #349](https://github.com/OpenSourceEconomics/respy/pull/349) into `respy`.

In [6]:
crit_func = rp.get_crit_func(params_base, options, df)

In [7]:
rslt, params_rslt = maximize(
    criterion=crit_func, 
    params=params_base, 
    algorithm="nlopt_bobyqa", 
    algo_options={"maxeval": 20},  
    # for the first tries we set the evaluations to a low number
    dashboard=False)

In [8]:
#params_rslt

### Working step 2: Ambiguity perturbation

We will disturb $\eta$ and subsequently estimate it to check whether the same values come out again.

More specifically:
* We will first set $\eta$ so the values presented at p.17 in Eisenhauer and Suchy (2020), i.e. baseline (0.01) and high (0.02)
* We will simulate the model with the set values
* We will perturb our starting values in the estimation and analyze whether they return to their initial values

In [9]:
eta_values = {
    "risk_only": 0.00,
    "baseline": 0.01,
    "high": 0.02,
}

In [10]:
params_eta = params_base.copy()
params_eta.loc[("eta", "eta"), :] = eta_values["baseline"]
params_eta

value
category                name                          
delta                   delta                     0.95
wage_fishing            exp_fishing               0.10
nonpec_fishing          constant                 -1.00
nonpec_hammock          constant                  2.50
                        not_fishing_last_period  -1.00
shocks_sdcorr           sd_fishing                1.00
                        sd_hammock                1.00
                        corr_hammock_fishing     -0.20
lagged_choice_1_hammock constant                  1.00
inadmissibility_penalty inadmissibility_penalty -20.00
eta                     eta                       0.01

Initialize the base parameter constraints. Base refers here to the case where all parameters are fixed.

In [11]:
#constr = [{"query": "category != 'eta'", "type": "fixed"}]
constr_base = [
    {"loc": "shocks_sdcorr", "type": "sdcorr"}, 
    {"loc": "eta", "type": "fixed"},
    {"loc": "delta", "type": "fixed"},
    {"loc": "wage_fishing", "type": "fixed"},
    {"loc": "nonpec_fishing", "type": "fixed"},
    {"loc": "nonpec_hammock", "type":"fixed"},
    {"loc": "shocks_sdcorr", "type": "fixed"},
]

We will keep all parameters fixed, but let $\eta$ vary during the estimation.

In [12]:
constr_eta = constr_base.copy()
constr_eta.remove({"loc": "eta", "type": "fixed"})

In [13]:
# Check the parameter combinations
eta_perturbed = {
    "true": [],
    "estimated": [],
    "yos": []
}

**Get a simulated data set** where agents act accordingly to $\eta = 0.01$.

In [14]:
simulate_eta = rp.get_simulate_func(params_eta, options)
df_eta = simulate_eta(params_eta)
#df_eta

In [15]:
for eta in np.linspace(0, 0.25, 11):
    
    # We want a change in the starting values (not in the simulated data)
    params_eta.loc[("eta", "eta"), :] = eta
    
    # params["group"] = params.index.get_level_values('category')
    # need above only if we want dashboard output
    
    # Get the criterion function with simulated data, but different param values
    crit_func_eta = rp.get_crit_func(params_eta, options, df_eta) 

    results, params_rslt = maximize(
        crit_func_eta, 
        params_eta,
        "nlopt_bobyqa",
        algo_options={"maxeval": 20},
        # In the "real world simulation" we will set this to a much higher number
        constraints=constr_eta
    )
    
    eta_perturbed["true"].append(eta)
    eta_perturbed["estimated"].append(params_rslt.loc["eta", "value"][0])
    #eta_perturbed["yos"].append(stat)
    
    # To do: write a wrapper for this thing

In [16]:
eta_perturbed

{'true': [0.0,
  0.025,
  0.05,
  0.07500000000000001,
  0.1,
  0.125,
  0.15000000000000002,
  0.17500000000000002,
  0.2,
  0.225,
  0.25],
 'estimated': [0.0,
  0.025,
  0.05,
  0.07500000000000001,
  0.1,
  0.125,
  0.15000000000000002,
  0.17500000000000002,
  0.2,
  0.225,
  0.25],
 'yos': []}

### Working step 3: Risk-only model

Write explicitly what is fixed.


We estimate a model with *risk-only*. This means we have $\eta = 0$ fixed for the simulated (estimation).
We want to see how different the likelihood is at the estimated parameter values.
We treat the risk-parameter as only unknown parameter.

In [17]:
params_risk_only = params_base.copy()
params_risk_only.loc[("eta", "eta"), :] = eta_values["risk_only"]
params_risk_only

value
category                name                          
delta                   delta                     0.95
wage_fishing            exp_fishing               0.10
nonpec_fishing          constant                 -1.00
nonpec_hammock          constant                  2.50
                        not_fishing_last_period  -1.00
shocks_sdcorr           sd_fishing                1.00
                        sd_hammock                1.00
                        corr_hammock_fishing     -0.20
lagged_choice_1_hammock constant                  1.00
inadmissibility_penalty inadmissibility_penalty -20.00
eta                     eta                       0.00

In [18]:
constr_risk_only = constr_base.copy()
constr_risk_only.remove({"loc": "delta", "type": "fixed"})

In [19]:
simulate_risk_only = rp.get_simulate_func(params_risk_only, options)
df_risk_only = simulate_risk_only(params_risk_only)

In [20]:
crit_func_risk_only = rp.get_crit_func(params_risk_only, options, df_risk_only)
crit_func(params_risk_only)

-6.803133136863534

In [21]:
results, params_rslt = maximize(
    crit_func, 
    params_risk_only,
    "nlopt_bobyqa",
    algo_options={"maxeval": 40},
    constraints=constr_risk_only,
    dashboard=False
)

In [22]:
params_rslt

value  lower  upper  \
category                name                                               
delta                   delta                     0.947039   -inf    inf   
wage_fishing            exp_fishing               0.100000   -inf    inf   
nonpec_fishing          constant                 -1.000000   -inf    inf   
nonpec_hammock          constant                  2.500000   -inf    inf   
                        not_fishing_last_period  -1.000000   -inf    inf   
shocks_sdcorr           sd_fishing                1.000000   -inf    inf   
                        sd_hammock                1.000000   -inf    inf   
                        corr_hammock_fishing     -0.200000   -inf    inf   
lagged_choice_1_hammock constant                  1.011116   -inf    inf   
inadmissibility_penalty inadmissibility_penalty -19.996838   -inf    inf   
eta                     eta                       0.000000   -inf    inf   

                                                          group  \
category                name                                      
delta                   delta                    All Parameters   
wage_fishing            exp_fishing              All Parameters   
nonpec_fishing          constant                 All Parameters   
nonpec_hammock          constant                 All Parameters   
                        not_fishing_last_period  All Parameters   
shocks_sdcorr           sd_fishing               All Parameters   
                        sd_hammock               All Parameters   
                        corr_hammock_fishing     All Parameters   
lagged_choice_1_hammock constant                 All Parameters   
inadmissibility_penalty inadmissibility_penalty  All Parameters   
eta                     eta                      All Parameters   

                                                                                            name  \
category                name                                                                       
delta                   delta                                                        delta_delta   
wage_fishing            exp_fishing                                     wage_fishing_exp_fishing   
nonpec_fishing          constant                                         nonpec_fishing_constant   
nonpec_hammock          constant                                         nonpec_hammock_constant   
                        not_fishing_last_period           nonpec_hammock_not_fishing_last_period   
shocks_sdcorr           sd_fishing                                      shocks_sdcorr_sd_fishing   
                        sd_hammock                                      shocks_sdcorr_sd_hammock   
                        corr_hammock_fishing                  shocks_sdcorr_corr_hammock_fishing   
lagged_choice_1_hammock constant                                lagged_choice_1_hammock_constant   
inadmissibility_penalty inadmissibility_penalty  inadmissibility_penalty_inadmissibility_penalty   
eta                     eta                                                              eta_eta   

                                                 _fixed_value  \
category                name                                    
delta                   delta                             NaN   
wage_fishing            exp_fishing                       0.1   
nonpec_fishing          constant                         -1.0   
nonpec_hammock          constant                          2.5   
                        not_fishing_last_period          -1.0   
shocks_sdcorr           sd_fishing                        1.0   
                        sd_hammock                        1.0   
                        corr_hammock_fishing             -0.2   
lagged_choice_1_hammock constant                          NaN   
inadmissibility_penalty inadmissibility_penalty           NaN   
eta                     eta                               0.0   

                                             

Working Step 4:
- Simulate data with $eta \neq$ 0 and estimate $\eta = 0$

### Summary of working steps

* We estimated a model by maximum likelihood using all true values as the starting point.

* We estimated a model with only $\eta$ perturbed.

* We estimated a model with risk-only, $\eta = 0$ fixed, to see how different the likelihood is at the estimated parameter values.